In [1]:
# Run this first!!!
from IPython.display import display, HTML

from __future__ import division
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('../common')
import util
import sampling_misc

# Goal

The goal now is to find $\mu_X$ and $\mu_Y$ for the size model. It seems like $\mu_X$ should be the easiest so we start there.

## A Subproblem

A good place to potentially start is looking at the probability that any one state will have a child that is a new thread. Let $X_{i, n}$ take the value of 1 if the $i^{th}$ state in depth $n$ has a child that is a new thread and 0 otherwise.

Recall that to find the failure of a child, we look at the parent's failure and see if we can advance in the DFA from that point. If not, we keep following the "failure chain" until we can find a point in which we can go forward. Therefore, we know we have a new thread if we have to follow the failure chain all the way back to the root state and then we are able to go forward from this point.

From this it becomes clear that the number of failures in the failure chain plays an important role. As such let $F_{i, n}$ be the number of failures in the failure chain for the $i^{th}$ state in depth $n$. For now let's assume some arbitrary value for this to see if we can derive the probability for this without having to worry about this.

For some alphabet $\Sigma$ we let $p_\sigma$ be the probability of seeing letter $\sigma$ in one of the sets in the generalized string $G$. With this established let's think about the aforementioned way that we can tell that something will be a new thread. Really we can think about this whole thing as a (slightly altered) geometric distribution since each time we go to another link in the failure chain we see if we can go forwards and stop if we can. This is slightly altered since the geometric can only take on finite values ($ \leq F_{i, n}$). This actually won't change anything because all of the other mass will be concentrated at some other point and can be ignored. So something has a new thread if for some $\sigma \in \Sigma$ the slightly altered geometric random variable is equal to $F_{i, n}$. That is...

$$
P(X_{i, n} | F_{i, n} = f) = \sum_{A \subseteq \Sigma} \left(1 - P\left(\textrm{There is no $\sigma \in \Sigma$ such that } Geom(p_\sigma) = f\right) \right) P(A)
$$

$$
= \sum_{A \subseteq \Sigma} \left(1 - \prod_{\sigma \in A} \left(1 - (1 - p_\sigma)^{m - 1} p_\sigma \right) \right) P(A)
$$

Where

$$
P(A) = \prod_{\sigma \in A} p_\sigma \prod_{\sigma \in \Sigma \setminus A} (1 - p_\sigma)
$$

Simulating this below, we see that this looks like the correct expression.

In [2]:
def get_theoretical(probs, m):
    return _expand_possibilities(probs, m)

def _expand_possibilities(probs, m, curr=None, index=0):
    if curr is None:
        curr = [False for _ in range(len(probs))]
    if index == len(probs):
        return _get_theor_term(probs, m, curr)
    nxt = list(curr)
    nxt[index] = True
    return (_expand_possibilities(probs, m, nxt, index + 1)
            + _expand_possibilities(probs, m, curr, index + 1))

def _get_theor_term(probs, m, subset):
    if True not in subset:
        return 0
    prob_f = 1
    prob_x = 1
    for i, include in enumerate(subset):
        p = probs[i]
        if include:
            prob_f *= p
            prob_x *= (1 - (1 - p) ** (m - 1) * p)
        else:
            prob_f *= (1 - p)
    return (1 - prob_x) * prob_f

def compare(probs, m, num_samples, depth):
    theor = get_theoretical(probs, m)
    df = sampling_misc.sample_state_has_new_thread(num_samples, probs, depth, [m])
    sampled = df['has_new_thread'].mean()
    print 'Theoretical: %f' % theor
    print 'Sampled: %f' % sampled
    print 'Relative Error: %f' % (abs(sampled - theor) / theor)

In [5]:
# PARAMETERS
PROBS = [0.5 for _ in range(4)] # Probability of seeing each letter.
NUM_SAMPLES = 100
DEPTH = 20 # Depth to check state at
M = 2 # Size of parent

compare(PROBS, M, NUM_SAMPLES, DEPTH)

Theoretical: 0.413818
Sampled: 0.409091
Relative Error: 0.011424


## What is the Distribution of $F_{i, n}$

If we can find the probability distribution for $F_{i, n}$ then we can find the probability distribution of $X_{i, n}$ through the total law of probability.

My first attempt at finding this is through a Markov Chain. Instead of indexing by $i$, let's consider that there is only one thread that we are following. Therefore we are looking at the random variable $F_n$. Furthermore, note that $F_n$ is closely tied to $F_{n - 1}$ since we rely on the parent state to determine what the child's failure transition is. This suggests that we should be looking at a first order Markov Chain.

What is $P\left(F_n = a | F_{n - 1} = b\right)$? First I assert that this probability is 0 if $a > b + 1$. This can be proved via induction. Note that $F_1 = 1$ deterministically and that the most that $F_2$ can be is 2. Now assume that this is true for all states up to and including depth $n = k$. For $F_{k + 1}$ consider the following scenario where the $F_k = b$. The best we can do is go one previous link in the chain ($b - 1$) before matching with some parent's child. From the inductive hypothesis we see that this can have at most one greater ($b$). Adding a failure function to the newly added child we get $b + 1$.

Finally note that there is a recursive aspect to this probability. If we want to find that the length of the failure chain is $a$, this means that we must consider the probability that the previous link in the chain had length $a - 1$. Therefore I think that the distribution is the following:

$$
P\left(F_n = a | F_{n - 1} = b\right) = \begin{cases} 
      0 & a > b + 1 \\
      \alpha & a = 1, b = 1 \\
      \beta & a = 2, b = 1 \\
      \sum_{\ell = 1}^{b - 1} P\left(F_n = a - 1 | F_{n - 1} = b - L\right) P(L = \ell) & \textrm{else}
   \end{cases}
$$

Here $L$ is an altered geometric random variable that describes how far down we fall before we are able to match with something.

Because this Markov Chain is irreducible we know that there is at most one stationary distribution $\pi$. However, we cannot guarantee that there is at least one because the state space is infinite. That being said, I am pretty sure that there exists one. If we were able to find such a $\pi$ then we would have the probabilities needed to finish the calculation we started in the first part.